# MEDCohort visualisation

## Import modules and patient data

In [ ]:
import datetime
import pickle

from MEDclasses import *
from src.semi_front.MEDcohort_utils import *
from src.semi_front.MEDprofiles_utils import *
import matplotlib as mpl
import numpy as np

%matplotlib qt

In [ ]:
# Get MEDProfiles
data_file = open('../data/mimic/MEDprofileData', 'rb')
# data_file = open('../data/meningioma/MEDprofileData', 'rb')
MEDprofile_list = pickle.load(data_file)[:10]
data_file.close()
cohort = MEDcohort(list_MEDprofile=MEDprofile_list)
df_cohort = cohort.profile_list_to_df()

In [ ]:
display(df_cohort)

## Define plot functions

In [ ]:
# Define global attributes
dict_selected_points = {}
dict_selected_annotations = {}
dict_points = {}
dict_annotations = {}
dict_class_time_points = {}
dict_bin_points = {}
dict_bin_annotations = {}
dict_figure_profile = {}
legend_points = []
xaxis = [FIXED_COLUMNS[1]]

In [ ]:
def button_pressed(event):
    """
    Called when a button pressed event occurs.
    :param event:
    :return:
    """
    # If the button pressed is the left MouseButton we display the data of
    # the nearest point in the axis
    if event.button == mpl.backend_bases.MouseButton.LEFT:
        display_attributes_values(event, df_cohort)

    # If the button pressed is the right MouseButton we highlight the patient
    # points in all the plot (for the nearest patient)
    elif event.button == mpl.backend_bases.MouseButton.RIGHT:
        display_patient_id_and_data(event, axes, df_cohort, classes_attributes_dict, dict_points, dict_annotations,
                                    dict_selected_points, dict_selected_annotations, xaxis)
    fig.canvas.draw()

In [ ]:
def button_pressed_in_simple_figure(event):
    """
    Called when a button is pressed on figure where there is one annotation per point.
    :param event:
    :return:
    """
    if event.button == mpl.backend_bases.MouseButton.LEFT or event.button == mpl.backend_bases.MouseButton.RIGHT:
        display_annotations(event)
        plt.gcf().canvas.draw_idle()

In [ ]:
def key_pressed_in_profile(event):
    """
    Called when a key is pressed on profile figure.
    :param event:
    :return:
    """
    # Set relative time at the class matching the axis in which the event occurs
    if event.key == 'r':
        figure_dict = dict_figure_profile[plt.gcf().number][1]
        r_pressed_in_profile(event, figure_dict['profile_axes'], figure_dict['profile_df'], classes_attributes_dict, figure_dict['profile_xaxis'], figure_dict['profile_points'], figure_dict['profile_annotations'])
        center_data_in_profile_plot(figure_dict['profile_points'])
        plt.gcf().canvas.draw_idle()

In [ ]:
def on_close_figure(event):
    """
    Remove profile data from figure profile dict.
    :param event:
    :return:
    """
    dict_figure_profile.pop(plt.gcf().number, None)

In [ ]:
def key_pressed(event):
    """
    Called when a key pressed event occurs.
    :param event:
    :return:
    """
    # Delete selected patients while delete key is pressed
    if event.key == 'delete':
        delete_pressed(dict_points, dict_selected_points, dict_selected_annotations, title)
    # Set relative time at the class matching the axis in which the event occurs
    elif event.key == 'r':
        r_pressed(event, axes, df_cohort, classes_attributes_dict, dict_points, dict_annotations, dict_selected_points, dict_selected_annotations, xaxis)
    # Set time points
    #elif event.key == 't':
        #t_pressed(event, axes, df_cohort, classes, all, dict_points, dict_annotations, dict_selected_points, dict_selected_annotations, xaxis, #dict_class_time_points, legend_points, fig)
    # Open bin figure
    elif event.key == 'y' or event.key == 'm':
        if event.key == 'y':
            frequency = 'year'
        else:
            frequency = 'month'
        bin_pressed(df_cohort, classes_attributes_dict, frequency, subplot_height, plot_width, colors, dict_bin_points, dict_bin_annotations, button_pressed_in_simple_figure)
    # Open selected profiles
    elif event.key == 'p':
        p_pressed(df_cohort, classes_attributes_dict, subplot_height, plot_width, xaxis, colors, dict_selected_points, dict_figure_profile, button_pressed_in_simple_figure, key_pressed_in_profile, on_close_figure)
    fig.canvas.draw()

In [ ]:
def on_close(event):
    """
    Called when the figure is closed.
    :param event:
    :return:
    """
    # Update the cohort dataframe according to the plot figure
    update_cohort(df_cohort, dict_points, dict_selected_points)

## Display Cohort

In [ ]:
# Define figure parameters
subplot_height = 1
plot_width = 10
classes_attributes_dict = {'demographic': ([], 'compact'), 'labevent': (['sodium_max', 'sodium_min', 'sodium_trend'], 'complete'), 'nrad': (['attr_0', 'attr_1', 'attr_2'], 'compact'), 'vp': ([], 'compact')}
# classes_attributes_dict = {'Demographic': ([], 'compact'), 'Therapy': ([], 'compact'), 'Pathology': ([], 'complete'), 'Event': ([], 'complete')}
colors = mpl.colormaps['Blues'](np.linspace(0, 1, len(classes_attributes_dict.keys()) + 1))

In [ ]:
# Create and display figure
fig, axes = set_plot(subplot_height, plot_width, classes_attributes_dict, colors)
fig.canvas.mpl_connect('button_press_event', button_pressed)
fig.canvas.mpl_connect('key_press_event', key_pressed)
fig.canvas.mpl_connect('close_event', on_close)
plt.xlabel("Date", fontsize=12)
display_cohort(axes, df_cohort, classes_attributes_dict, dict_points, dict_annotations, xaxis)
title = fig.suptitle(f'MEDcohort composed by {len(set(df_cohort.index))} patients', fontsize=16)